In [17]:
import numpy as np
import keras
from keras import layers, models

In [2]:
def reweight_distribution(original_dist, temperature=0.5):
    distribution = np.log(original_dist) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

In [5]:
path = keras.utils.get_file('nietzsche.txt', 
                            origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()
len(text)

In [8]:
maxlen = 60
step = 3
sentences = []
next_chars = []

In [9]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])

In [11]:
chars = sorted(list(set(text)))
char_indices = dict((char, chars.index(char)) for char in chars)

In [14]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

In [18]:
model = models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [19]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

In [20]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = reweight_distribution(preds, temperature)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

In [38]:
np.random.multinomial(1, [1./3] * 3, 1)

array([[0, 0, 1]])

In [39]:
import sys, random

In [40]:
for epoch in range(1, 60):
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index : start_index + maxlen]
    
    for temperature in [0.2, 0.5, 1., 1.2]:
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[i, t, char_indices[char]] = 1.
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(pred, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)

Epoch 1/1
200278/200278 [==============================] - 222s 1ms/step - loss: 2.0011
ere right--have not all gods
hitherto been such sanctified, 

TypeError: data type not understood